# Using age, cholestrol, maximum heart rate and resting blood pressure to classify heart disease patients from Cleveland #

**John Mao, Caitlyn Chan and Ritaksha Roy Burman**


### Introduction ###
Cardiovascular heart diseases are the leading cause of death worldwide, accounting for an estimated 17.9 million casualties each year according to the [WHO.](https://www.who.int/health-topics/cardiovascular-diseases#tab=tab_1)

Our project seeks to use age, cholesterol, maximum heart rate and resting blood pressure to diagnosis individuals with coronary artery disease, (CAD). CAD occurs when the arteries that deliver blood to the heart is reduced in diameter due to plaque buildup (CDC, 2021).

The goal of this project is to classify patients of whether they have CAD for preventatitive interventions.

The question we wish to answer is: 
**What is the likelihood a patient will have coronary artery disease based on age, sex, cholestrol levels, resting blood pressure and maximum heart rate achieved?**

To answer this, we are using the [heart disease](https://archive.ics.uci.edu/dataset/45/heart+disease) database.

We are then comparing this to the Switzerland [switzerland_heart_disease](https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data) database.

|variable | units | desription |
|-------- | ----- | ---------- |
| chol | mg/dl | serum cholestrol |
| age | years | N/A |
| trestbps | mmHg | resting blood pressure (on admission to hospital) |
| num | N/A | 0 = unafflicted by disease, (1,2,3,4) showing presense of heart disease. |
| thalach | N/A | maximum heart rate achieved (bpm) |
| diagnosis | N/A | Diagnosis will be true or false relating to num |


### Methods ### - TO BE ALTERTED TO FIT OUR REPORT 
1. PReprocessing data
2. finding best k value, visualizing the best k, underfitting/over fitting
3. testing our classifier
4. switzerland
    
We will be using variables from the Heart Disease Database, in particular the Cleaveland database:
- chol : serium cholestoral in mg/dl
- age : age in years
- trestbps : resting blood pressure (in mm Hg on admission to hospital)
- sex
- num : 0 = unafflicted by disease, (1,2,3,4) showing presense of heart disease.
- thalach: maximum heart rate achieved (bpm) 

To tidy the data, we have changed the sex and num (presense of heart disease) to factors. **num** uses integers 1 to 4 to determine diagnosis of CAD, thus we have created a new column which sets presence of heart disease (**num** > 0) to TRUE and absence (**num** = 0) to FALSE. 


The dataframe has been split into a training and testing set. The data is stratified by the categorical variable, **diagnosis**. 

To visualize the results of our data analysis, scatterplots of different numerical variables were plotted against eachother. 

* In the plot of **trestbp vs age**

A distinct region of resting blood pressure is high in the ages of 50 to 70. 

* In the plot of **chol vs age**

Women in the ages of 55-65 appear to be at risk whereas men appear to be at risk from ages 45 - 70.

* In the plot of **thalach vs chol**

There are distinct regions of TRUE and FALSE, and a mix of both. 

These relationships and distribution determined selection of **trestbp, age, thalach and chol** as predictors. 

We are going to train and classify using K-nearest neighbours to determine if a new patient is likely to be afflicted by heart disease.

### Expected Outcomes and Significance ##
We expect to see a positive trend with resting blood pressure and age. We also expect to see that on average, those  with heart diseases will have higher blood pressures.
We wish to explore how sex impacts an individual's potential risk.

This data anaylsis could be advantageous to enhance timely diagnosis of cardiovascular disease to help implement interventions and recognize risk factors. 

Future questions this could lead to: 
1. Which of the factors have the greatest influence on diagnosis of CAD?
2. What are some preventative measures one can take to minimize risk of heart disease?


### Bibliography ### 

Detrano, R., Janois, A., Steinbrunn, W., Pfisterer, Matthias., Schmid, Johann-Jakob., Sandhu, S., Guppy, KH., Lee, S., Froelicher, V. (1988). International Application of a New Probability Algorithm for the Diagnosis of Coronary Artery Disease. *The American Journal of Cardiology*, 64(1), 304-309.  https://doi.org/10.24432/C52P4X.

Coronary Artery Disease. (2021, July 19). Centers for Disease Control and Prevention (CDC). https://www.cdc.gov/heartdisease/coronary_ad.htm


Janosi, A ., Steinbrunn, W., Pfisterer, M., and Detrano, R. (1988). Heart Disease. UCI Machine Learning Repository. https://doi.org/10.24432/C52P4X.

World Health Organization. (n.d.). https://www.who.int/health-topics/cardiovascular-diseases#tab=tab_1. 




    

### Preliminary exploratory data analysis ###
We have 303 total rows of data amongst our 5 variables with no observations that contain missing data.



**Importing Libraries**

In [ ]:
# importing libraries 
library(tidyverse)
library(tidymodels)
library(repr)
library(dplyr)
library(RColorBrewer)
install.packages("caret")
library(caret)
install.packages("Information")
library(Information)
# formatting tables
options(repr.matrix.max.rows = 6)

**Importing and reading the data set from the web**

In [ ]:
# cleveland heart data
heart_data <- read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")
# switzerland data
switzerland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data", col_names = FALSE)


**Cleaning and wrangling the data and splitting the data into training and testing sets**

#To add minimal explanation later 

In [ ]:
# set the seed 
set.seed(1)

# cleaning and wrangling the data frame, selecting relevant columns
heart_data_clean <- heart_data |>
select(chol, trestbps, age, sex, thalach, num) |>
mutate(diagnosis = as.factor(ifelse(is.na(num), NA, (num>0)))) |>
mutate(sex = as_factor(sex))|>
mutate(sex = fct_recode(sex, "Male" = "1", "Female" = "0")) |>
mutate(num = as_factor(num)) 

# cleaning and wrangling switzerland data frame

colnames(switzerland_data) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

switzerland_data_clean <- switzerland_data |>
select(chol, trestbps, age, sex, thalach, num) |>
mutate(diagnosis = ifelse(is.na(num), NA, (num>0))) |>
mutate(sex = as_factor(sex))|>
mutate(sex = fct_recode(sex, "Male" = "1", "Female" = "0")) |>
mutate(num = as_factor(num)) 

# splitting data into training data set and testing data set 
heart_split <- initial_split(heart_data_clean, prop = 0.75, strata = num)

heart_testing <- testing(heart_split)
heart_training <- training(heart_split)

heart_training
print(paste("Total rows", nrow(heart_data_clean)))
print(paste("Rows with missing data", sum(is.na(heart_data_clean))))



**Summarizing the Training Data**

Before we visualize and begin our prediction analysis, we need to check that the two predictor classes are balanced so that one class prediction is not favored over the other.

In [ ]:
heart_summary_table <- heart_training |>
                    group_by(diagnosis) |>
                    summarize(count = n()) |>
                    arrange(-count)
heart_summary_table

Based off the above summary table, the counts for TRUE and FALSE heart disease diagnoses are roughly balanced in the training set. 

**Visualizing the relationships between our chosen predictor variables and the class variable**
#short explanation later 

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5)
# Plot the training data

heart_plot1 <- heart_training |> 
            ggplot(aes(x= age, y= trestbps, color = diagnosis)) +
            geom_point()+ 
            labs(x= "Age", y = "Resting Blood Pressure \n (mm Hg)", fill = "Diagnosis of Heart Disease", color = "Diagnosis of Heart Disease", title = "Resting Blood Pressure versus Age", num = "Diagnosis of Heart Disease")+ # rename false and true 
            theme(text = element_text(size = 12))+
            theme(plot.title = element_text(hjust = 0.5))
heart_plot1


heart_plot2 <- heart_training |>
mutate(num = ifelse(num == 0, FALSE, TRUE)) |>
ggplot(aes(x = age, y = chol, color = num)) + 
  geom_point() +
  labs(num = "Diagnosis of Heart Disease",
       color = "Diagnosis of Heart Disease",
       title = "Diagnosis of Heart Disease versus Cholesterol (mg/dl) and Age",
       y = "Cholesterol (mg/dl)", x= "Age (years)") +
    theme(text = element_text(size = 13)) +
    scale_size_discrete(range = c(1, 2)) +
    facet_grid(cols = vars(sex))+
    theme(plot.title = element_text(hjust = 0.5))


heart_plot2

heart_plot3 <- heart_training |> 
            ggplot(aes(x= chol, y= thalach, color = diagnosis)) +
            geom_point()+ 
            labs(x= "Cholestrol (mg/dl)", y = "Maximum Heart Rate (bpm)", fill = "Diagnosis of Heart Disease", color = "Diagnosis of Heart Disease", title = "Resting Blood Pressure versus Age", num = "Diagnosis of Heart Disease")+ # rename false and true 
            theme(text = element_text(size = 12))+
            theme(plot.title = element_text(hjust = 0.5))
heart_plot3

# change plot to thalac and age 


**Finding the best k value**
#add explanation of steps later
#visualzing accuracy using the accuracy vs knn plot -- ask TA about this maybe we can do a smoothness color plot to check for under/over fitting

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

# Create a recipe with the chosen predictor variables, standardize
heart_recipe <- recipe(diagnosis ~ chol + thalach + trestbps + age, data = heart_training) |>
                step_scale(all_predictors()) |>
                step_center(all_predictors())

# create a K-nearest neighbor model specification with tuning on the number of neighbors 
spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
        set_engine("kknn") |>
        set_mode("classification") 

#10-fold cross validation 
heart_vfold <- vfold_cv(heart_training, v = 10, strata = diagnosis)
gridvals <- tibble(neighbors = c(1:190))


# create a workflow analysis, then collect metrics
knn_results <- workflow() |>
                add_recipe(heart_recipe) |>
                add_model(spec) |>
                tune_grid(resamples = heart_vfold, grid = gridvals) |>
                collect_metrics()  

# filter metrics for accuracy 
accuracies_lots <- knn_results |>
                filter(.metric == "accuracy") |>
                select(neighbors, mean) |>
                arrange(-mean)

# Visualize k accuracy 
cross_val_plot <- accuracies_lots |> # accuracies
                ggplot(aes(x= neighbors, y = mean))+
                geom_point() +
                geom_line() +
                labs(x= "Neighbors", y = "Accuracy Estimate")+
                ggtitle("Cross Validation Accuracy vs Number of Neighbors")+
                theme(text = element_text(size = 15)) 


        

cross_val_plot

accuracies_lots

**testing the classifier w/ confusion matrix, determining accuracy/recall/precision, testing our classifier against the majority classifier**
#add explanation later 

In [ ]:
best_k <- accuracies_lots |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)

spec_best_k <- nearest_neighbor(weight_func = "rectangular", neighbors = 23) |>
        set_engine("kknn") |>
        set_mode("classification") 

knn_best_results <- workflow() |>
                add_recipe(heart_recipe) |>
                add_model(spec_best_k) |>
                fit(data = heart_training)

heart_predictions <- predict(knn_best_results, heart_testing) |>
                    bind_cols(heart_testing) |>
                    select(diagnosis, .pred_class)

heart_conf_mat <- heart_predictions |>
                conf_mat(truth = diagnosis, estimate = .pred_class)

# conf_table <- as.table(heart_conf_mat)
heart_major <- heart_training |>
            group_by(diagnosis) |>
            summarise(n=n()) |>
            mutate(percent = 100*n/nrow(heart_training))

heart_major

# heart_acc <- sum(diag(conf_table))/ sum(conf_table)
# heart_acc
heart_conf_mat
heart_predictions
# MAKE SURE TO CALCULATE THE ACCURACY, RECALL, PRECISION
best_k

heart_best_k <- workflow() |>
        add_recipe(heart_recipe) |>
        add_model(spec) |>
        tune_grid(resamples = heart_vfold, grid = gridvals) |>
        collect_metrics() |>
        filter(.metric == "accuracy") |>
        select(neighbors, mean) |>
        mutate(mean = 100*mean)|>
        arrange(desc(mean))

head(heart_best_k)

In [ ]:
#checking consistency of our classifier using an additional data set from the data base
#switzerland 
# cross compare the cleveland dataset to the swiss one to see if the model is consistent 

#load tidied swiss data 

discussion section to address
- answering/how we answered the question (predictor/class variables) 
- expected findings compared to outcome and plausible reasoning for this (faridas notes) and relate to classifiers 68% accuracy... and if the classifier is good for cleaveland as it is for switzerland 
- impact of findings
- future questions 
- bibliography  

In [ ]:
heart_training_IV <- heart_training |>
            mutate(diagnosis = ifelse(diagnosis == TRUE, 1, 0))
heart_training_IV
IV <- create_infotables(data = heart_training_IV, y = "diagnosis")
IV$Summary
WOE_table <- IV$Tables
WOE_table